# Import Bibliotheken

In [552]:
# Festlegung des Device
import platform

# Laden der Daten
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Operationen mit Texten
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

# Modell
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Evaluierung
import torchmetrics

# Auswertung der Evaluation
import pandas as pd
import altair as alt
import numpy as np

from sklearn import preprocessing

### Package Versionen

In [553]:
for package in [torchtext,torch, torchmetrics, spacy, alt, pd]:
    print(f'{package.__name__}: {package.__version__}')

torchtext: 0.13.1
torch: 1.12.1
torchmetrics: 0.9.3
spacy: 3.4.2
altair: 4.2.0
pandas: 1.4.3


# Device Auswahl

In [554]:
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Daten

## Dataset

### Erstellung Dataset

In [555]:
comment_df = pd.read_excel('../data/clean/Google_Rezensionen.xlsx')
comment_df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


In [556]:
class TeamGoogleBewertungenDataSet(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = comment_df.iloc[index]['Kommentar']
        category = comment_df.iloc[index]['Kategorien']
        return text, category

### Bestimmung Test- und Traningsdaten

In [557]:
# Bestimmung der Länge der Test- und Traningsdaten
dataset = TeamGoogleBewertungenDataSet(comment_df)
len_train = round(len(dataset)* 0.7)
len_test = round(len(dataset) * 0.3)
assert len(comment_df) == len_train + len_test

In [558]:
# Zufällige Aufsplittung der Anzahl von Test- und Trainingsdaten
train_set, test_set = torch.utils.data.random_split(dataset, [len_train, len_test]) 

In [559]:
# next(iter(train_set))

## Dataloader

### Tokenisierung

In [560]:
tokenizer = get_tokenizer(
    'spacy', 
    language='de_core_news_lg')

In [561]:
# Beispiel für Anwendung des tokenizer
# sentence = "This isn't a very long example."
# tokenizer(sentence)

### Erstellung Vokabular

In [562]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [563]:
# Länge des Trainungsets
len(train_set)

2798

In [564]:
tokens = yield_tokens(train_set)

In [565]:
next(iter(tokens))

['große',
 'tankstelle',
 'mit',
 'reichhaltiger',
 'angebotspalette',
 'kaminholz',
 'tankstellenüblich',
 'sehr',
 'teuer']

In [566]:
vocab = build_vocab_from_iterator(tokens, min_freq=2, specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

In [567]:
len(vocab)

2460

In [568]:
vocab(['Hallo', 'sehr', 'Personal'])

[0, 3, 0]

In [569]:
vocab.lookup_token(0)

'<unk>'

### Encoding

In [570]:
onehot = torch.zeros(1,len(vocab))

In [571]:
pos = vocab(['nette', 'Bedienung', 'sauber'])
pos

[31, 0, 44]

In [572]:
def encode(text, vocab):
    tokens = tokenizer(text)
    onehot = torch.zeros(1,len(vocab))
    onehot[:,vocab(tokens)] = 1
    return onehot

### Multilabel Encoding

In [573]:
import torch
import torch.nn

In [574]:
df = comment_df.explode('Kategorien')
df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


#### Kategorien splitten

In [575]:
einzelLabels = comment_df['Kategorien'].str.split(',')
label_list_export = einzelLabels.to_list()
label_list_export

[['Personal', 'Shop', 'Bistro'],
 ['Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal'],
 ['Pricing', 'DigitalFueling'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing', 'Kraftstoffauswahl'],
 ['Personal', 'Öffnungszeiten'],
 ['Personal', 'SB-Waschboxen', 'Waschanlage'],
 ['Shop'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing'],
 ['Bistro'],
 ['Personal', 'Pricing', 'Shop', 'Waschanlage'],
 ['Personal'],
 ['Personal'],
 ['Bistro'],
 ['Erscheinungsbild'],
 ['Personal', 'Pricing'],
 ['Personal'],
 ['Personal'],
 ['Personal', 'Shop'],
 ['Erscheinungsbild'],
 ['Personal', 'Bistro'],
 ['Erscheinungsbild', 'Nacht-/Tankautomat', 'Waschanlage', 'Staubsauger'],
 ['Pricing'],
 ['Personal'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal'],
 ['Bistro'],
 ['Pricing', 'Kraftstoffauswahl'],
 ['Pricing'],
 ['Nacht-/Tankautomat'],
 ['Shop', 'Bistro'],
 ['Erscheinungsbild', 'Personal', 'Sanitär

##### In der nachfolgenden Zelle wird aus der verschachtelten Liste eine "flache Liste"

In [576]:
flat_list = []
for sublist in label_list_export:
    for item in sublist:
        flat_list.append(item)

flat_list

['Personal',
 'Shop',
 'Bistro',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'DigitalFueling',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Kraftstoffauswahl',
 'Personal',
 'Öffnungszeiten',
 'Personal',
 'SB-Waschboxen',
 'Waschanlage',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Bistro',
 'Personal',
 'Pricing',
 'Shop',
 'Waschanlage',
 'Personal',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Pricing',
 'Personal',
 'Personal',
 'Personal',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Nacht-/Tankautomat',
 'Waschanlage',
 'Staubsauger',
 'Pricing',
 'Personal',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Bistro',
 'Pricing',
 'Kraftstoffauswahl',
 'Pricing',
 'Nacht-/Tankautomat',
 'Shop',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Sanitär',
 'Bistro',
 'Pricing',
 'Verkehrsanbindung'

##### Doppelte Werte werden entfernt, so dass jeder Eintrag nur einmalig vorhanden ist. Dies erreichen wir mit set()

In [577]:
flat_set = set(flat_list)
flat_set

{'AdBlue',
 'Bistro',
 'DigitalFueling',
 'E-Fuels',
 'E-Mobilität',
 'Erscheinungsbild',
 'Kartenakzeptanzen',
 'Kraftstoffauswahl',
 'Luft',
 'Nacht-/Tankautomat',
 'Paketservice',
 'Personal',
 'Pricing',
 'SB-Waschboxen',
 'Sanitär',
 'Shop',
 'Staubsauger',
 'Tankpool',
 'Verkehrsanbindung',
 'WLAN',
 'Waschanlage',
 'Werkstatt',
 'Zapfsäulen',
 'Öffnungszeiten'}

In [578]:
unflat = [[x] for x in flat_set]

In [579]:
label_vocab = build_vocab_from_iterator(unflat, min_freq=1,)

label_vocab.set_default_index(vocab["<unk>"])

In [580]:
label_vocab.lookup_token(3)

'E-Fuels'

#### Dictionary erstellen

In [581]:
label_dict = {label:i for i, label in enumerate(flat_set)}
label_dict

{'Personal': 0,
 'Zapfsäulen': 1,
 'Paketservice': 2,
 'E-Fuels': 3,
 'Luft': 4,
 'Bistro': 5,
 'AdBlue': 6,
 'Kraftstoffauswahl': 7,
 'Staubsauger': 8,
 'Sanitär': 9,
 'DigitalFueling': 10,
 'WLAN': 11,
 'E-Mobilität': 12,
 'Waschanlage': 13,
 'Öffnungszeiten': 14,
 'Kartenakzeptanzen': 15,
 'Erscheinungsbild': 16,
 'Werkstatt': 17,
 'Shop': 18,
 'Verkehrsanbindung': 19,
 'SB-Waschboxen': 20,
 'Nacht-/Tankautomat': 21,
 'Pricing': 22,
 'Tankpool': 23}

#### Tensor

In [582]:
doc2 = label_list_export[2]
doc2

['Personal', 'Waschanlage']

In [583]:
pos_vals = [label_dict[val] for val in doc2]
pos_vals

[0, 13]

In [584]:
labels = torch.LongTensor(pos_vals)
labels

tensor([ 0, 13])

In [585]:
y_onehot = nn.functional.one_hot(labels, num_classes=24)
y_onehot

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [586]:
y_onehot = y_onehot.sum(dim=0).float()
y_onehot

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])

In [587]:
# labels = torch.tensor([1,5,8,0])
# labels = labels.unsqueeze(0)
# target = torch.zeros(labels.size(0), 24).scatter_(1, labels, 1.)
# print(target)

### Erstellung Dataloader

In [588]:
# label_pipeline = lambda x: int(x) -1

In [589]:
def collate_batch(batch):
    label_list, text_list = [], []
 
    for (_text,_label) in batch:
    
        # Vorverarbeitung der Label
        # label_list.append(label_pipeline(_label))
        processed_labels = encode(_label, label_vocab)

        label_list.append(processed_labels)

        # Vorverarbeitung der Texte
        processed_text = encode(_text, vocab)
        
        # Zusammenführen sämtlicher Textrepräsentationen in einer Liste
        text_list.append(processed_text)
 
    # Zusammenführen aller Label in einem Tensor
    labels = torch.cat(label_list, dim=0)
    
    # Verbinden der Tensoren in text_list zu einem Tensor
    texts = torch.cat(text_list, dim = 0)

    # Ausgabe der Texte und der Label
    return texts.to(device), labels.to(device)

In [590]:
train_loader = DataLoader(
 train_set, batch_size=64,
 shuffle=True,
 collate_fn=collate_batch,
 num_workers=0
)

## Architektur und Training

### Architektur

In [591]:
class LinearTextClassificationModel(nn.Module):

    def __init__(self, vocab_size, num_class):
        super(LinearTextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(vocab_size, 200)
        self.fc2 = nn.Linear(200, num_class)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = x #F.softmax(x, dim=1)
        return output

In [592]:
vocab_size = len(vocab)
num_class = flat_set.__len__()
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

### Training

In [593]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.03

## Initialisierung Fehlerfunktion
loss_fn = nn.CrossEntropyLoss()

## Initialisierung Fehlerfunktion
bce_loss = nn.BCEWithLogitsLoss()

## Initialisierung Optimizer
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate, momentum=0.9)

## Definition der Epochen
num_epochs = 150

In [594]:
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold= 0.5).to(device)

loss_hist = {}
accuracy_hist = {}

In [595]:
for (text, label) in train_loader:
    #print(label[[0][0]])
    #print(text)
    labelnew = label.type(torch.LongTensor)
    print(labelnew[[0][0]])

tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0,

In [596]:
# Training
model.train()

for epoch in range(num_epochs):

    # Dokumentation Loss -> Erkennung ob Netz konvergiert
    running_loss = 0.0
    num_batches = 0

    for (text, label) in train_loader:
        num_batches += 1
        pred = model(text)

        # Der Fehler wird berechnet
        loss = bce_loss(pred, label)
        # Der Fehler wird über das Netz zurückpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        ## Bestimmung der Accuracy für den Batch
        train_accuracy(pred, label.type(torch.LongTensor))
        
        # running loss
        running_loss +=loss.item()
 
    loss_hist[epoch] = running_loss/num_batches
 
    batch_train_accuracy = train_accuracy.compute()
    print(f"Training Accuracy for epoch {epoch}:{batch_train_accuracy}")
    accuracy_hist[epoch] = batch_train_accuracy.cpu().item()
    train_accuracy.reset()

Training Accuracy for epoch 0:0.0
Training Accuracy for epoch 1:0.0017869906732812524
Training Accuracy for epoch 2:0.23659756779670715
Training Accuracy for epoch 3:0.31629735231399536
Training Accuracy for epoch 4:0.3195139467716217
Training Accuracy for epoch 5:0.31987133622169495
Training Accuracy for epoch 6:0.3195139467716217
Training Accuracy for epoch 7:0.3195139467716217
Training Accuracy for epoch 8:0.3195139467716217
Training Accuracy for epoch 9:0.3184417486190796
Training Accuracy for epoch 10:0.3177269399166107
Training Accuracy for epoch 11:0.3173695504665375
Training Accuracy for epoch 12:0.3159399628639221
Training Accuracy for epoch 13:0.3137955665588379
Training Accuracy for epoch 14:0.3109363913536072
Training Accuracy for epoch 15:0.3091493844985962
Training Accuracy for epoch 16:0.303788423538208
Training Accuracy for epoch 17:0.30343103408813477
Training Accuracy for epoch 18:0.29878485202789307
Training Accuracy for epoch 19:0.2969978451728821
Training Accuracy 

In [597]:
batch_train_accuracy

tensor(0.6190)

### Auswertung Fehlerentwicklung

In [598]:
loss_df = pd.DataFrame.from_dict(loss_hist, orient= 'index').reset_index()

loss_df.columns = ['Epoch', 'Loss']
loss_df.head()

,Epoch,Loss
0,0,0.648962
1,1,0.508384
2,2,0.329729
3,3,0.250910
4,4,0.231148


In [599]:
train_chart = alt.Chart(loss_df).mark_line().encode(
    x=alt.X('Epoch', title = 'Anzahl Epochen'),
    y=alt.Y('Loss', title = 'Mittlerer Fehler')
).properties(title='Trainingsfehlerentwicklung in Abhängigkeit von der Anzahl der Epochen')
train_chart

alt.Chart(...)

### Auswertung Genauigkeit

In [600]:
accuracy_df = pd.DataFrame.from_dict(accuracy_hist, orient = 'index').reset_index()
accuracy_df.columns = ['Epoch', 'Accuracy']
accuracy_df.head()

,Epoch,Accuracy
0,0,0.000000
1,1,0.001787
2,2,0.236598
3,3,0.316297
4,4,0.319514


In [601]:
accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x=alt.X('Epoch',title = 'Anzahl Epochen'),
    y=alt.Y('Accuracy', title = 'Genauigkeit')
).properties(title='Trainingsgenauigkeit in Abhängigkeit von der Anzahl der Epochen')
accuracy_chart

alt.Chart(...)

## Evaluierung

In [602]:
model.eval()

LinearTextClassificationModel(
  (fc1): Linear(in_features=2460, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=24, bias=True)
)

### Gesamtevaluierung

In [603]:
test_loader = DataLoader(test_set, batch_size= 64, shuffle=True,collate_fn=collate_batch)

In [604]:
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold=0.5).to(device)

with torch.no_grad():
    for (text, label) in test_loader:
        # Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()

In [605]:
total_valid_accuracy

tensor(0.5346)

### Evaluierung nach Klassen

In [606]:
# Evaluieren
valid_accuracy = torchmetrics.Accuracy(
    average=None,
    num_classes=flat_set.__len__(),
    # subset_accuracy = True
).to(device)

with torch.no_grad():
    for(text, label) in test_loader:
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()
    total_valid_accuracy

In [607]:
total_valid_accuracy # Werte sind zu hoch,da vermutlich eine Nichzuordnung als Zuordnung interpretiert wird

tensor([0.8282, 0.9616, 0.9900,    nan, 0.9992, 0.9608, 0.9691, 0.9541, 0.9850,
        0.9791, 0.9958, 0.9616, 0.9408, 0.9858, 0.9733, 0.9491, 0.9850, 0.9975,
        0.9683,    nan, 0.9516, 0.9900, 0.9883, 0.9691])

### Evaluierung nach der Anzahl der Epochen

In [608]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [609]:
test_loader = DataLoader(test_set, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [610]:
vocab_size = len(vocab)
num_class = flat_set.__len__()
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

In [611]:
bce_loss = nn.BCEWithLogitsLoss()

In [612]:
learning_rate = 0.03

optimizer = optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9)

In [613]:
num_epochs = 150

In [614]:
# Evaluierung wurd vorbereitet
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold=0.5).to(device)
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True, threshold=0.5).to(device)

train_accuracy_hist = {}
train_loss_hist = {}

valid_accuracy_hist = {}
valid_loss_hist = {}

In [615]:
for epoch in range(num_epochs):

    # Loss wird dokumentiert, sodass zu erkenne ist, ob das Netz konvergiert
    num_train_batches = 0
    num_eval_batches = 0
    train_running_loss = 0.0
    eval_running_loss = 0.0

    model.train()
    for (text, label) in train_loader:
        num_train_batches += 1
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        # Der Fehler wird berechnet
        # Als Ausgabe wird ein Tensor zurückgegeben
        loss = bce_loss(pred, label)
        # Der Fehler wird über das Netz zurükpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        # Aktualisierung des States der train_accurracy
        train_accuracy.update(pred, label.type(torch.LongTensor))
        # running loss
        train_running_loss += loss.item()

    train_loss_hist[epoch] = train_running_loss / num_train_batches

    # Berechnung der Trainingsgenauigkeit für die Epoche
    total_train_accuracy = train_accuracy.compute()
    # Hinzufügen der Trainingsgenauigkeit zu dem Dictionary
    train_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    # Validierung
    model.eval()
    with torch.no_grad():
        for (text, label) in test_loader:
            num_eval_batches += 1
            # Die Vorhersage wird für das Model erzeugt
            pred = model(text)
            # Test loss
            eval_loss = bce_loss(pred, label)
            # Running loss
            eval_running_loss += eval_loss.item()
            # Aktualisierung des States der valid_accuracy
            valid_accuracy.update(pred, label.type(torch.LongTensor))
    
    valid_loss_hist[epoch] = eval_running_loss/num_eval_batches

    # Berechnung der Validierungsgenauigkeit für die Epoche
    total_valid_accuracy = valid_accuracy.compute()
    # Hinzufügen der Validierungsgenauigkeit zum Dictionary
    valid_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    print(f"Training Accuracy for epoch {epoch}: {total_train_accuracy}")
    print(f"Validation Accuracy for epoch {epoch}: {total_valid_accuracy}")
    train_accuracy.reset()
    valid_accuracy.reset()



Training Accuracy for epoch 0: 0.0
Validation Accuracy for epoch 0: 0.0
Training Accuracy for epoch 1: 0.018584704026579857
Validation Accuracy for epoch 1: 0.1392827332019806
Training Accuracy for epoch 2: 0.28770551085472107
Validation Accuracy for epoch 2: 0.31693077087402344
Training Accuracy for epoch 3: 0.3137955665588379
Validation Accuracy for epoch 3: 0.3202669024467468
Training Accuracy for epoch 4: 0.3191565275192261
Validation Accuracy for epoch 4: 0.32193493843078613
Training Accuracy for epoch 5: 0.31879913806915283
Validation Accuracy for epoch 5: 0.3202669024467468
Training Accuracy for epoch 6: 0.31879913806915283
Validation Accuracy for epoch 6: 0.3194328546524048
Training Accuracy for epoch 7: 0.3191565275192261
Validation Accuracy for epoch 7: 0.3202669024467468
Training Accuracy for epoch 8: 0.31879913806915283
Validation Accuracy for epoch 8: 0.3177648186683655
Training Accuracy for epoch 9: 0.3173695504665375
Validation Accuracy for epoch 9: 0.3177648186683655
Tr

In [616]:
train_loss_df = pd.DataFrame.from_dict(train_loss_hist, orient = "index").reset_index()
train_loss_df["type"] = "train"
valid_loss_df = pd.DataFrame.from_dict(valid_loss_hist, orient = "index").reset_index()
valid_loss_df["type"] = "test"
loss_df = pd.concat([train_loss_df,valid_loss_df])
loss_df.columns = ["Epoch", "Loss", "Type"]

In [617]:
train_eval_loss_chart = alt.Chart(loss_df).mark_line().encode(
    x=alt.X('Epoch',title = 'Anzahl Epochen'),
    y=alt.Y('Loss', title='Mittlerer Fehler'),
    color=alt.Color('Type',title='Typ')
).properties(title='Fehlerentwicklung in Abhängigkeit von der Anzahl der Epochen')
train_eval_loss_chart

alt.Chart(...)

In [618]:
valid_accuracy_hist

{0: array(0., dtype=float32),
 1: array(0.13928273, dtype=float32),
 2: array(0.31693077, dtype=float32),
 3: array(0.3202669, dtype=float32),
 4: array(0.32193494, dtype=float32),
 5: array(0.3202669, dtype=float32),
 6: array(0.31943285, dtype=float32),
 7: array(0.3202669, dtype=float32),
 8: array(0.31776482, dtype=float32),
 9: array(0.31776482, dtype=float32),
 10: array(0.31693077, dtype=float32),
 11: array(0.3144287, dtype=float32),
 12: array(0.3144287, dtype=float32),
 13: array(0.31276062, dtype=float32),
 14: array(0.3119266, dtype=float32),
 15: array(0.30942452, dtype=float32),
 16: array(0.30525437, dtype=float32),
 17: array(0.30525437, dtype=float32),
 18: array(0.30442035, dtype=float32),
 19: array(0.30191827, dtype=float32),
 20: array(0.30191827, dtype=float32),
 21: array(0.29941618, dtype=float32),
 22: array(0.29858217, dtype=float32),
 23: array(0.2969141, dtype=float32),
 24: array(0.29357797, dtype=float32),
 25: array(0.2910759, dtype=float32),
 26: array(0

In [619]:
train_accuracy_hist

{0: array([0.82819015, 0.9616347 , 0.98999166,        nan, 0.99916595,
        0.96080065, 0.96914095, 0.95412844, 0.9849875 , 0.9791493 ,
        0.9958299 , 0.9616347 , 0.940784  , 0.98582155, 0.97331107,
        0.9491243 , 0.9849875 , 0.9974979 , 0.9683069 ,        nan,
        0.95162636, 0.98999166, 0.9883236 , 0.96914095], dtype=float32),
 1: array(0., dtype=float32),
 2: array(0.13928273, dtype=float32),
 3: array(0.31693077, dtype=float32),
 4: array(0.3202669, dtype=float32),
 5: array(0.32193494, dtype=float32),
 6: array(0.3202669, dtype=float32),
 7: array(0.31943285, dtype=float32),
 8: array(0.3202669, dtype=float32),
 9: array(0.31776482, dtype=float32),
 10: array(0.31776482, dtype=float32),
 11: array(0.31693077, dtype=float32),
 12: array(0.3144287, dtype=float32),
 13: array(0.3144287, dtype=float32),
 14: array(0.31276062, dtype=float32),
 15: array(0.3119266, dtype=float32),
 16: array(0.30942452, dtype=float32),
 17: array(0.30525437, dtype=float32),
 18: array(0

In [620]:
# Erste Epoche löschen, da diese nicht neu initialisiert wird und Daten aus vorherigen Schritten enthält

del train_accuracy_hist[0]
del valid_accuracy_hist[0]

In [621]:
train_accuracy_df = pd.DataFrame.from_dict(train_accuracy_hist, orient = "index").reset_index()
train_accuracy_df["type"] = "train"
valid_accuracy_df = pd.DataFrame.from_dict(valid_accuracy_hist, orient = "index").reset_index()
valid_accuracy_df["type"] = "test"
accuracy_df = pd.concat([train_accuracy_df,valid_accuracy_df])
accuracy_df.columns = ["Epoch", "Accuracy", "Type"]

In [622]:
accuracy_df

,Epoch,Accuracy,Type
0,1,0.000000,train
1,2,0.139283,train
2,3,0.316931,train
3,4,0.320267,train
4,5,0.321935,train
...,...,...,...
144,145,0.532110,test
145,146,0.529608,test
146,147,0.532944,test
147,148,0.532944,test


In [623]:
train_eval_accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
 x=alt.X('Epoch', title='Anzahl der Epochen'),
 y=alt.Y('Accuracy', title='Genauigkeit'),
 color=alt.Color('Type',title='Typ')
).properties(title='Genauigkeiten des Trainings- und Testdatensatzes nach der Anzahl der Epochen')
train_eval_accuracy_chart

alt.Chart(...)